In [8]:
from RawPreProcessing import rawpreprocessing
import Data_Extract
from SlidingWindow import slidingwindow
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, TensorDataset
"""# 클래스 인스턴스 생성
processor = rawpreprocessing(data_set_per_label=10, time_window=3) # label당 10개 , 창의 크기 3
num_data_set=processor.num_data_set
# 데이터 처리 및 시각화 반복
for i in range(1, num_data_set + 1):
    file_path = f"C:/Users/전재형/OneDrive/바탕 화면/raw_data/rawdatas/Raw Data{i}.csv"
    
    # CSV 파일에서 양 끝 10% 데이터 제거
    result_df = processor.remove_edges_from_csv(file_path)

    # 데이터 시각화 & T초로 자르기
    processor.plot_csv_data(result_df)

    # 새로운 CSV 파일로 저장
    #trimmed_file_path = f"C:/Users/교육생-PC08/trimmed_data/trimmed_data{i}.csv"
    #result_df.to_csv(trimmed_file_path, index=False)

# 최종 3차원 배열 생성
data_set=processor.make_total_array()
Y_label=processor.Y_label
num=len(data_set)"""

'# 클래스 인스턴스 생성\nprocessor = rawpreprocessing(data_set_per_label=10, time_window=3) # label당 10개 , 창의 크기 3\nnum_data_set=processor.num_data_set\n# 데이터 처리 및 시각화 반복\nfor i in range(1, num_data_set + 1):\n    file_path = f"C:/Users/전재형/OneDrive/바탕 화면/raw_data/rawdatas/Raw Data{i}.csv"\n    \n    # CSV 파일에서 양 끝 10% 데이터 제거\n    result_df = processor.remove_edges_from_csv(file_path)\n\n    # 데이터 시각화 & T초로 자르기\n    processor.plot_csv_data(result_df)\n\n    # 새로운 CSV 파일로 저장\n    #trimmed_file_path = f"C:/Users/교육생-PC08/trimmed_data/trimmed_data{i}.csv"\n    #result_df.to_csv(trimmed_file_path, index=False)\n\n# 최종 3차원 배열 생성\ndata_set=processor.make_total_array()\nY_label=processor.Y_label\nnum=len(data_set)'

In [9]:
data_set=np.load('C:/Users/전재형/Motion_ML/full_data.npy')
num=len(data_set)
Y_label=['handshaking', 'punching', 'waving', 'walking']

In [10]:
X=[]
y=[]
sliding_window_processor = slidingwindow(data_set, Y_label)
for j in range(0, num):  # row data 갯수 만큼 돌림
        part_data = data_set[j]

        # Fourier 변환을 통해 최대 주파수 구하기
        max_freq = sliding_window_processor.fourier_trans_max_amp(part_data[:, 3], 100)  # absolute 값
        #print(f"Max Frequency for dataset {j}: {max_freq}")

        # SlidingWindow 클래스 인스턴스 생성 및 슬라이딩 윈도우 처리
        win_datas=sliding_window_processor.sliding_window(1/max_freq,1/max_freq*0.5,j)
        #print(Data_Extract.data_extraction(win_datas[3]).extract_feature())
        for i in range(0, len(win_datas)):
                
                X.append(Data_Extract.data_extraction(win_datas[i]).extract_feature())
                y.append(Y_label[int(j/10)])


In [11]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X_tensor = torch.tensor(X, dtype=torch.float32)   # (batch_size, seq_length, input_dim)
y_tensor = torch.tensor(y_encoded, dtype=torch.long)

# Dataset & DataLoader 설정
# dateset을 n개로 나눠서 최적화 진행
batch_size = 16
dataset = TensorDataset(X_tensor, y_tensor)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False) # 시계열 데이터니깐 shuffle을 하면 안되지만 sliding window를 사용했기때문에 여기선 True



class GRUMotionClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(GRUMotionClassifier, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size) # 분류 문제이기 때문에 outputsize를 받음. 만약 regression문제라면 1로 고정정

    def forward(self, x):
        gru_out, hidden = self.gru(x)  # hidden: (num_layers, batch_size, hidden_size)
        # 만약 gru_out이 2차원 텐서라면, 시퀀스 길이가 1인 경우일 수 있음
        if gru_out.ndimension() == 2:  # (batch_size, hidden_size)일 때
            out = self.fc(gru_out)  # 바로 fc 레이어에 전달
        else:  # (batch_size, seq_length, hidden_size)일 때
            out = self.fc(gru_out[:, -1, :])  # 마지막 시점의 hidden state 사용
            
        return out
    
model = GRUMotionClassifier(input_size=40, hidden_size=64, num_layers=2, output_size=len(Y_label))
# input_size는 현재 x, y, z, a에서 뽑은 feature 40개
# hidden_size는 이전 데이터를 얼마나 기억할 것인지, 높으면 정확성이 올라가지만 너무 올라가면 과적합
# num_layers는 GRU 층
# output_size는 y_label의 개수(현재는 waving 밖에 없어서 1로함.)

In [12]:
# ========== 3. 학습 설정 ==========
learning_rate=0.001 # 학습률, weight를 update할때 얼만큼 weight를 조정할건지, 너무 크면 확확 바뀌고, 너무 작으면 찔끔찔끔 변화함. 적당한 것이 0.001
criterion = nn.CrossEntropyLoss()  # 분류 문제 -> CrossEntropyLoss
optimizer = optim.Adam(model.parameters(), lr=learning_rate) # 처음에는 큰 lr을 사용하다가 점차 작은 lr을 사용하는 최적화 알고리즘




# ========== 4. 학습 실행 ==========
num_epochs = 50

for epoch in range(num_epochs):
    for batch_X, batch_y in data_loader:
        optimizer.zero_grad() # 이전 Epoch에서 계산된 기울기(Gradient) 초기화
    
        # Forward
        outputs = model(batch_X) # 모델에 입력 데이터를 넣어 예측값 계산
        loss = criterion(outputs, batch_y)  # 손실(loss) 계산

        # Backward & Optimize
        loss.backward() # 역전파(Backpropagation) 수행하여 기울기 계산
        optimizer.step() # 가중치 업데이트

    if (epoch + 1) % 10 == 0: # 10번 주기로 학습이 잘되고 있는지 확인
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [10/50], Loss: 0.0658
Epoch [20/50], Loss: 0.0246
Epoch [30/50], Loss: 0.0164
Epoch [40/50], Loss: 0.0115
Epoch [50/50], Loss: 0.0077


In [13]:
test=np.load('C:/Users/전재형/Motion_ML/test_data.npy')
tests=[]
y_test=[]

sliding_window_test = slidingwindow(test, Y_label)
for j in range(0, len(test)):  # row data 갯수 만큼 돌림
        part_data = test[j]

        # Fourier 변환을 통해 최대 주파수 구하기
        max_freq = sliding_window_test.fourier_trans_max_amp(part_data[:, 3], 100)  # absolute 값
        #print(f"Max Frequency for dataset {j}: {max_freq}")

        # SlidingWindow 클래스 인스턴스 생성 및 슬라이딩 윈도우 처리
        win_datas=sliding_window_test.sliding_window(1/max_freq,1/max_freq*0.5,j)
        #print(Data_Extract.data_extraction(win_datas[3]).extract_feature())
        for i in range(0, len(win_datas)):
                
                tests.append(Data_Extract.data_extraction(win_datas[i]).extract_feature())
                y_test.append(Y_label[int(j/10)])
test_sample = torch.tensor(tests, dtype=torch.float32)

In [14]:
# ========== 5. 테스트 ==========
# test_sample = torch.randn(1, seq_length, input_dim)  # 임의의 테스트 데이터
model.eval()
with torch.no_grad():
    prediction = model(test_sample)
    predicted_class = torch.argmax(prediction, dim=1)
for i, pred in enumerate(predicted_class):
    print(f"Test Sample {i+1}: Predicted Motion = {pred.item()}")

Test Sample 1: Predicted Motion = 0
Test Sample 2: Predicted Motion = 2
Test Sample 3: Predicted Motion = 0
Test Sample 4: Predicted Motion = 0
Test Sample 5: Predicted Motion = 0
Test Sample 6: Predicted Motion = 0
Test Sample 7: Predicted Motion = 0
Test Sample 8: Predicted Motion = 0
Test Sample 9: Predicted Motion = 0
Test Sample 10: Predicted Motion = 0
Test Sample 11: Predicted Motion = 0
Test Sample 12: Predicted Motion = 0
Test Sample 13: Predicted Motion = 0
Test Sample 14: Predicted Motion = 0
Test Sample 15: Predicted Motion = 0
Test Sample 16: Predicted Motion = 0
Test Sample 17: Predicted Motion = 0
Test Sample 18: Predicted Motion = 0
Test Sample 19: Predicted Motion = 0
Test Sample 20: Predicted Motion = 0
Test Sample 21: Predicted Motion = 0
Test Sample 22: Predicted Motion = 0
Test Sample 23: Predicted Motion = 0
Test Sample 24: Predicted Motion = 0
Test Sample 25: Predicted Motion = 0
Test Sample 26: Predicted Motion = 0
Test Sample 27: Predicted Motion = 0
Test Sampl

In [15]:
len(y)

527